In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder\
                    .master("local")\
                    .appName("group_window")\
                    .getOrCreate()
                    


24/04/17 16:54:34 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
24/04/17 16:54:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/17 16:54:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.format("csv")\
                .option("header",True)\
                .option("inferschema",True)\
                .load("./sales_transactions.csv")
                
df.show()

+---+------------+------+-----------+
| ID|Product Name| Price|Total Value|
+---+------------+------+-----------+
|  1|      Jacket|872.19|    7849.71|
|  2|       Jeans|865.39|    1730.78|
|  3|         Hat|183.11|     549.33|
|  4|     Sweater|305.15|      610.3|
|  5|         Hat| 84.59|      845.9|
|  6|       Socks|191.72|    1533.76|
|  7|      Jacket|696.02|     696.02|
|  8|     Sweater|170.31|    1532.79|
|  9|       Jeans|163.04|    1141.28|
| 10|     Sweater| 990.3|     1980.6|
| 11|     Sweater| 92.88|     743.04|
| 12|       Shoes|306.33|    1531.65|
| 13|       Shirt| 65.67|     197.01|
| 14|      Jacket|555.63|    4445.04|
| 15|       Jeans| 48.99|     342.93|
| 16|      Jacket|323.87|    2590.96|
| 17|      Jacket| 895.2|     8952.0|
| 18|       Shoes|106.37|     531.85|
| 19|         Hat|146.77|     880.62|
| 20|       Socks|584.94|    5264.46|
+---+------------+------+-----------+
only showing top 20 rows



In [26]:
df1 = df.groupBy("ID").agg(sum(col("Total Value")).alias("total_value")).orderBy("ID")
df1.show()

+---+-----------+
| ID|total_value|
+---+-----------+
|  1|    7849.71|
|  2|    1730.78|
|  3|     549.33|
|  4|      610.3|
|  5|      845.9|
|  6|    1533.76|
|  7|     696.02|
|  8|    1532.79|
|  9|    1141.28|
| 10|     1980.6|
| 11|     743.04|
| 12|    1531.65|
| 13|     197.01|
| 14|    4445.04|
| 15|     342.93|
| 16|    2590.96|
| 17|     8952.0|
| 18|     531.85|
| 19|     880.62|
| 20|    5264.46|
+---+-----------+
only showing top 20 rows



In [38]:
# window_specs = Window.orderBy(col("total_value"),desc())
window_specs = Window.orderBy("total_value").desc()

AttributeError: 'WindowSpec' object has no attribute 'desc'

In [37]:
window_specs = Window.orderBy("total_value").desc()
df2 = df1.withColumn("rank", dense_rank().over(window_specs))
df2.show()

24/04/17 17:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+-----------+----+
| ID|total_value|rank|
+---+-----------+----+
|212|     9973.0|   1|
| 94|     9757.9|   2|
|148|     9648.3|   3|
|204|     9554.5|   4|
|471|     9547.8|   5|
|160|     9502.0|   6|
|626|     9435.7|   7|
|570|     9357.0|   8|
|355|     9286.9|   9|
|266|     9164.3|  10|
|856|     9077.4|  11|
|311|     9028.7|  12|
| 17|     8952.0|  13|
|725|     8948.2|  14|
|283|    8944.29|  15|
| 86|     8895.3|  16|
| 33|     8835.5|  17|
|142|    8808.66|  18|
|290|    8783.37|  19|
|850|    8774.55|  20|
+---+-----------+----+
only showing top 20 rows



In [41]:
window_specs = Window.orderBy(desc("total_value"))
df2 = df1.withColumn("rank", dense_rank().over(window_specs))
df2.show()


24/04/17 17:19:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+-----------+----+
| ID|total_value|rank|
+---+-----------+----+
|212|     9973.0|   1|
| 94|     9757.9|   2|
|148|     9648.3|   3|
|204|     9554.5|   4|
|471|     9547.8|   5|
|160|     9502.0|   6|
|626|     9435.7|   7|
|570|     9357.0|   8|
|355|     9286.9|   9|
|266|     9164.3|  10|
|856|     9077.4|  11|
|311|     9028.7|  12|
| 17|     8952.0|  13|
|725|     8948.2|  14|
|283|    8944.29|  15|
| 86|     8895.3|  16|
| 33|     8835.5|  17|
|142|    8808.66|  18|
|290|    8783.37|  19|
|850|    8774.55|  20|
+---+-----------+----+
only showing top 20 rows

